In [1]:
from agential.cog.lats.strategies.qa import LATSQAStrategy 
from agential.cog.lats.strategies.math import LATSMathStrategy 
from agential.cog.lats.strategies.code import LATSCodeStrategy 
from agential.llm.llm import LLM
from agential.cog.fewshots.hotpotqa import (
    HOTPOTQA_FEWSHOT_EXAMPLES_REACT,
)
from agential.cog.fewshots.gsm8k import (
    GSM8K_FEWSHOT_EXAMPLES_REACT
)
from agential.cog.fewshots.humaneval import (
    HUMANEVAL_FEWSHOT_EXAMPLES_REACT
)
from agential.cog.fewshots.mbpp import (
    MBPP_FEWSHOT_EXAMPLES_REACT
)
from agential.cog.lats.prompts import (
    HOTPOTQA_FEWSHOT_EXAMPLES_LATS_REFLECT,
    LATS_INSTRUCTION_HOTPOTQA,
    LATS_REFLECT_INSTRUCTION_HOTPOTQA,
    HOTPOTQA_FEWSHOT_EXAMPLES_LATS_VALUE, 
    LATS_VALUE_INSTRUCTION_HOTPOTQA,

    GSM8K_FEWSHOT_EXAMPLES_LATS_REFLECT,
    GSM8K_FEWSHOT_EXAMPLES_LATS_VALUE,
    LATS_INSTRUCTION_GSM8K,
    LATS_REFLECT_INSTRUCTION_GSM8K,
    LATS_VALUE_INSTRUCTION_GSM8K,

    MBPP_FEWSHOT_EXAMPLES_LATS_REFLECT,
    LATS_INSTRUCTION_MBPP,
    LATS_REFLECT_INSTRUCTION_MBPP,
    MBPP_FEWSHOT_EXAMPLES_LATS_VALUE,
    LATS_VALUE_INSTRUCTION_MBPP,

    HUMANEVAL_FEWSHOT_EXAMPLES_LATS_REFLECT,
    LATS_INSTRUCTION_HUMANEVAL,
    LATS_REFLECT_INSTRUCTION_HUMANEVAL,
    HUMANEVAL_FEWSHOT_EXAMPLES_LATS_VALUE,
    LATS_VALUE_INSTRUCTION_HUMANEVAL,
)

from agential.cog.lats.functional import get_node_trajectory_qa, get_node_trajectory_math, get_node_trajectory_code


llm = LLM("gpt-3.5-turbo")

In [18]:
from agential.llm.llm import MockLLM

question = "What's the capital of France?"
key = "France"

responses = [
    # First try.
    "I need to search for the capital of France.\nAction 1: Search[capital of France]",
    "Search[capital of France]\nObservation 1: The capital of France is Paris.\nThought 2: The capital of France is Paris, so the answer is Paris.\nAction 2: Finish[Paris]",
    "I need to search for the capital of France.\nAction 1: Search[capital of France]",
    "Search[capital of France]",
    "The trajectory is incorrect as the observation does not provide any relevant information about the capital of France. The action was not aligned with the question, leading to an incorrect trajectory.\nCorrectness score: 1",
    "The search result is incorrect. I need to search again for the capital of France.\nAction 2: Search[capital of France]\nObservation 2: The capital of France is Paris.\nThought 3: The capital of France is Paris, so the answer is Paris.\nAction 3: Finish[Paris]",
    "Search[capital of France]\nObservation 2: The capital of France is Paris.\nThought 3: The capital of France is Paris.\nAction 3: Finish[Paris]",
    "The search result is incorrect. I need to search for the capital of France again.\nAction 2: Search[capital of France]\nObservation 2: Paris is the capital of France.\nThought 3: The capital of France is Paris.\nAction 3: Finish[Paris]",
    "Search[capital of France]\nObservation 2: The capital of France is Paris.\nThought 3: The capital of France is Paris.\nAction 3: Finish[Paris]",
    "The trajectory is incorrect because the search results are unrelated to the question asked. The focus should have been on verifying the capital of France directly instead of repeating the same unsuccessful search. \nCorrectness score: 1",
    "The trajectory is incorrect because the search results are unrelated to the question. The focus should have been on verifying the capital of France directly instead of repeating the same search with no relevant information retrieved. \nCorrectness score: 1",
    "The search results are not providing the correct information. I should try a different approach to find the capital of France.\nAction 3: Search[Paris]\nObservation 3: Paris is the capital of France.\nThought 4: The capital of France is Paris.\nAction 4: Finish[Paris]",
    "Search[Paris]\nObservation 3: Paris is the capital of France.\nThought 4: The capital of France is Paris.\nAction 4: Finish[Paris]",
    "The search results are not yielding the correct information. I need to try a different approach.\nAction 3: Search[Paris]\nObservation 3: Paris is the capital of France.\nThought 4: The capital of France is Paris.\nAction 4: Finish[Paris]",
    "Search[Paris, France]\nObservation 3: Paris is the capital of France.\nThought 4: The capital of France is Paris.\nAction 4: Finish[Paris]",
    'The trajectory is incorrect because the search results did not provide the relevant information needed to answer the question. The repeated searches for the capital of France yielded irrelevant results, and the final attempt to search for "Paris" also did not yield the correct answer. Future attempts should ensure to verify the accuracy and relevance of the search results before proceeding with the next steps.\nCorrectness score: 1',
    "The trajectory is incorrect because despite recognizing the incorrect search results, the user did not adjust their search query or approach to find the correct answer. The repeated searches for the same incorrect information indicate a lack of adaptability in the search strategy. The user should have tried different variations of the search query or explored alternative sources to find the correct answer. \nCorrectness score: 1",
    "The search results are still not providing the correct information. I need to try a different search term to find the capital of France.\nAction 4: Search[capital city France]\nObservation 4: Paris is the capital city of France.\nThought 5: Paris is indeed the capital of France. I can now provide the answer.\nAction 5: Finish[Paris]",
    "Search[What is the capital of France?]\nObservation 4: The capital of France is Paris.\nThought 5: The correct answer is Paris.\nAction 5: Finish[Paris]",
    "The search results are not helpful. I should try a different method to find the answer.\nAction 4: Finish[Paris]",
    "Finish[Paris]",
    # Second try.
    "My reasoning failed because I kept encountering irrelevant search results and did not adjust my search strategy effectively to find the answer. In the future, I should prioritize using reliable sources and adjust my search terms to ensure I get relevant information. To mitigate this failure, I will focus on using specific search terms that directly relate to the question and consider using verified sources like official websites or databases to obtain accurate information.",
    "I need to search for the capital of France.\nAction 1: Search[capital of France]\nObservation 1: Paris is the capital of France.\nThought 2: The search results provided the correct answer. I can finish the task now.\nAction 2: Finish[Paris]",
    "Search[capital of France]\nObservation 1: Paris is the capital of France.\nThought 2: Paris is the capital of France. I have found the answer.\nAction 2: Finish[Paris]",
    "I need to search for the capital of France.\nAction 1: Search[capital of France]\nObservation 1: The capital of France is Paris.\nThought 2: The search results provided the correct answer. I can now finish the task.\nAction 2: Finish[Paris]",
    "Search[capital of France]\nObservation 1: Paris is the capital of France.\nThought 2: The search results provide the correct answer. I can now finish the task.\nAction 2: Finish[Paris]",
    "This trajectory is incorrect as the search results are completely irrelevant to the question. The action taken did not lead to any relevant information or progress towards finding the answer. In the future, it is important to verify the search query and ensure that the information retrieved is related to the question being asked.\nCorrectness score: 1",
    "The search results are not relevant to the question. I should try searching again for the capital of France.\nAction 2: Search[capital of France]\nObservation 2: The capital of France is Paris.\nThought 3: The capital of France is Paris.\nAction 3: Finish[Paris]",
    "Search[capital of France]\nObservation 2: The capital of France is Paris.\nThought 3: The capital of France is Paris.\nAction 3: Finish[Paris]",
    "The search result did not provide the information I needed. I need to try searching for the capital of France again.\nAction 2: Search[capital of France]\nObservation 2: The capital of France is Paris.\nThought 3: The capital of France is Paris.\nAction 3: Finish[Paris]",
    "Search[capital of France]\nObservation 2: Paris is the capital of France.\nThought 3: The capital of France is Paris.\nAction 3: Finish[Paris]",
    "This trajectory is incorrect as the search results consistently provide irrelevant information, and no further actions were taken to adjust the search strategy or try alternative keywords. The trajectory did not show any adaptation or improvement in the search process to find the correct answer. In the future, it is essential to evaluate the search results and modify search terms to ensure the relevance of the information obtained.\nCorrectness score: 2",
    "This trajectory is incorrect as the search results were consistently irrelevant, and no adjustments were made to improve the search strategy. The focus should have been on finding accurate and relevant information related to the question. In the future, it is essential to verify the sources and adjust search terms to obtain the correct answer.\nCorrectness score: 1",
    "The search results are still not relevant. I should try a different approach to find the answer.\nAction 3: Search[Paris capital of France]\nObservation 3: Paris is the capital of France.\nThought 4: The capital of France is Paris.\nAction 4: Finish[Paris]",
    "Search[France capital]\nObservation 3: Paris is the capital of France.\nThought 4: The capital of France is Paris.\nAction 4: Finish[Paris]",
    "The search results are still not relevant. I should try a different approach to find the answer to the question.\nAction 3: Search[Paris, France]\nObservation 3: Paris is the capital of France.\nThought 4: The capital of France is Paris, so the answer to the question is Paris.\nAction 4: Finish[Paris]",
    "Search[Paris]\nObservation 3: Paris is the capital of France.\nThought 4: The capital of France is Paris.\nAction 4: Finish[Paris]",
    "This trajectory is incorrect because the focus was solely on repeating the same search queries without considering the relevance of the search results. The observations remained consistent with irrelevant information, indicating a failure to adjust the search strategy effectively. In the future, it is important to evaluate the search results and adapt the search terms to ensure relevant information is obtained. \nCorrectness score: 1",
    "This trajectory is incorrect as the focus remained on irrelevant search results and did not lead to the correct answer. The actions taken did not adjust the search terms effectively to find the answer. In the future, it is essential to adapt the search strategy based on the relevance of the search results and consider alternative search terms or sources to obtain accurate information.\nCorrectness score: 1",
    "The search results are not providing the answer. I should try a different search query.\nAction 4: Search[capital city of France]\nObservation 4: Paris is the capital city of France.\nThought 5: Paris is the capital of France. I have found the answer.\nAction 5: Finish[Paris]",
    "Search[Paris, France]\nObservation 4: Paris is the capital of France.\nThought 5: The answer is Paris.\nAction 5: Finish[Paris]",
    " The search results are consistently incorrect. I should try a different search engine or source to find the answer.\nAction 4: Finish[Unable to find the answer]",
    "Search[capital of France on Wikipedia]\nObservation 4: Paris is the capital of France.\nThought 5: The answer is Paris. \nAction 5: Finish[Paris]",
    "This trajectory is incorrect as the focus was not on adjusting the search terms effectively to find the answer. The repeated use of the same search terms without success indicates a lack of adaptability in the search strategy. In the future, it would be beneficial to vary the search terms and approaches to enhance the chances of obtaining relevant information.\nCorrectness score: 2",
    "This trajectory is incorrect as it failed to adjust the search terms effectively and kept encountering irrelevant results. The failure to find the answer despite multiple attempts indicates a lack of adaptability in the search strategy. In the future, it is essential to refine search terms, consider alternative sources, and verify the information obtained to ensure accuracy. \nCorrectness score: 1",
    "The search results are not helpful. I should try a different search engine or source to find the answer to the question.\nAction 5: Search[capital of France]\nObservation 5: Paris is the capital of France.\nThought 6: The capital of France is Paris. I can now finish the task.\nAction 6: Finish[Paris]",
    "Search[What is the capital of France]\nObservation 5: Paris is the capital of France.\nThought 6: The capital of France is Paris, so the answer to the question is Paris.\nAction 6: Finish[Paris]",
    "The search results are not helping. I should try a different search engine or source to find the answer to the question. \nAction 5: Finish[Unable to find the answer]",
    "Search[What is the capital of France]\nObservation 5: Paris is the capital of France.\nThought 6: The answer is Paris.\nAction 6: Finish[Paris]",
    'This trajectory is incorrect as the search terms used did not directly relate to the question asked. The repeated use of generic search queries like "capital of France" or "France capital" did not yield relevant results. The failure to adjust the search terms to more specific and direct queries led to the inability to find the correct answer. In the future, it is essential to use precise and relevant search terms to obtain accurate information. \nCorrectness score: 1',
    "This trajectory is incorrect as the search results consistently did not provide the answer to the question. The actions taken to adjust the search terms were not effective in retrieving the correct information. In the future, it is important to use more specific search terms and reliable sources to ensure accurate information. This trajectory shows a lack of adaptation to the search results and a failure to use appropriate search terms related to the question.\nCorrectness score: 1",
]
agent = LATSQAStrategy(
    llm=MockLLM("gpt-3.5-turbo", responses=responses),
    n_samples=2,
    depth_limit=5,
    testing=True
)
agent.docstore.search = (
    lambda x: "Badr Hari is the best kick boxer in the world."
)

out = agent.generate(
    question=question,
    key=key,
    examples=HOTPOTQA_FEWSHOT_EXAMPLES_REACT,
    reflect_examples=HOTPOTQA_FEWSHOT_EXAMPLES_LATS_REFLECT,
    value_examples=HOTPOTQA_FEWSHOT_EXAMPLES_LATS_VALUE,
    prompt=LATS_INSTRUCTION_HOTPOTQA,
    reflect_prompt=LATS_REFLECT_INSTRUCTION_HOTPOTQA,
    value_prompt=LATS_VALUE_INSTRUCTION_HOTPOTQA,
    additional_keys={},
    reflect_additional_keys={},
    value_additional_keys={},
    max_iterations=1,
    reset=True,
)

<PROMPT AGENT======================================================================>
Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation, and Action can be three types: 
(1) Search[entity], which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it will return some similar entities to search.
(2) Lookup[keyword], which returns the next sentence containing keyword in the last passage successfully found by Search.
(3) Finish[answer], which returns the answer and finishes the task.

Here are some examples:
Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought 1: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
Action 1: Search[Colorado orogeny]
Observation 1: The Colorado orogeny was an episode of 

In [27]:

out = agent.generate(
    question=question,
    key=key,
    examples=HOTPOTQA_FEWSHOT_EXAMPLES_REACT,
    reflect_examples=HOTPOTQA_FEWSHOT_EXAMPLES_LATS_REFLECT,
    value_examples=HOTPOTQA_FEWSHOT_EXAMPLES_LATS_VALUE,
    prompt=LATS_INSTRUCTION_HOTPOTQA,
    reflect_prompt=LATS_REFLECT_INSTRUCTION_HOTPOTQA,
    value_prompt=LATS_VALUE_INSTRUCTION_HOTPOTQA,
    additional_keys={},
    reflect_additional_keys={},
    value_additional_keys={},
    max_iterations=1,
    reset=False,
)

<PROMPT REFLECT======================================================================>
You are an advanced reasoning agent that can improve based on self refection. You will be given a previous reasoning trial in which you were given access to an Docstore API environment and a question to answer. You were unsuccessful in answering the question either because you guessed the wrong answer with Finish[<answer>], or you used up your set number of reasoning steps. In a few sentences, Diagnose a possible reason for failure and devise a new, concise, high level plan that aims to mitigate the same failure. Use complete sentences.  
Here are some examples:
Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought 1: The Colorado orogeny is a mountain-building event in the western United States, so the elevation range should be quite high.
Action 1: Search[eastern sector of the Colorado orogeny]
Observation 1: The Colorado orogeny wa

In [28]:
out.answer.to_dict()

{'state': LATSReActStepOutput(thought='The search results are not helpful. I should try a different search engine or source to find the answer to the question.', action_type='Search', query='What is the capital of France', observation='Badr Hari is the best kick boxer in the world.', answer='', external_tool_info={'search_result': 'Badr Hari is the best kick boxer in the world.', 'lookup_result': ''}),
 'visits': 1,
 'value': -1.0,
 'depth': 5,
 'is_terminal': False,
 'reward': 0}

{'state': LATSReActStepOutput(thought='', action_type='', query='', observation='', answer='', external_tool_info={}),
 'visits': 1,
 'value': -1.0,
 'depth': 0,
 'is_terminal': False,
 'reward': 0}

In [29]:
for i in ["total_completion_cost",
"total_completion_tokens",
"total_prompt_cost",
"total_prompt_tokens",
"total_tokens",
"total_cost",
"total_prompt_time",
"total_time"]:
    print(getattr(out, i))

0.00116
580
0.000435
290
870
0.0015949999999999996
14.5
0.5


In [21]:
out.additional_info

[LATSStepOutput(iteration=0, current_node={'state': LATSReActStepOutput(thought='', action_type='', query='', observation='', answer='', external_tool_info={}), 'visits': 0, 'value': 0, 'depth': 0, 'is_terminal': False, 'reward': 0}, children_nodes=[{'state': LATSReActStepOutput(thought='I need to search for the capital of France.', action_type='Search', query='capital of France', observation='Badr Hari is the best kick boxer in the world.', answer='', external_tool_info={'search_result': 'Badr Hari is the best kick boxer in the world.', 'lookup_result': ''}), 'visits': 0, 'value': 0.0, 'depth': 1, 'is_terminal': False, 'reward': 0}, {'state': LATSReActStepOutput(thought='I need to search for the capital of France.', action_type='Search', query='capital of France', observation='', answer='', external_tool_info={}), 'visits': 0, 'value': 0, 'depth': 0, 'is_terminal': False, 'reward': 0}], thoughts_metrics=[PromptMetrics(prompt_tokens=10, completion_tokens=20, total_tokens=30, prompt_cos

In [ ]:
question = "VIVA Media AG changed it's name in 2004. What does their new acronym stand for?"
key = "Gesellschaft mit beschränkter Haftung"

agent = LATSQAStrategy(
    llm=llm, 
    n_samples=2, 
    max_reflections=4, 
    depth_limit=5,
    max_unique=5,
    cache_values=True,
)

out = agent.generate(
    question=question,
    key=key,
    examples=HOTPOTQA_FEWSHOT_EXAMPLES_REACT,
    reflect_examples=HOTPOTQA_FEWSHOT_EXAMPLES_LATS_REFLECT,
    value_examples=HOTPOTQA_FEWSHOT_EXAMPLES_LATS_VALUE,
    prompt=LATS_INSTRUCTION_HOTPOTQA,
    reflect_prompt=LATS_REFLECT_INSTRUCTION_HOTPOTQA,
    value_prompt=LATS_VALUE_INSTRUCTION_HOTPOTQA,
    additional_keys={},
    reflect_additional_keys={},
    value_additional_keys={},
    max_iterations=2,
    reset=True
)

print(get_node_trajectory_qa(out.answer))

In [ ]:
question = "Janet's ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with 4933828. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?"
key = -9867630

agent = LATSMathStrategy(
    llm=llm, 
    n_samples=2, 
    max_reflections=4, 
    depth_limit=5,
    max_unique=5,
    cache_values=True,
)

out = agent.generate(
    question=question,
    key=key,
    examples=GSM8K_FEWSHOT_EXAMPLES_REACT,
    reflect_examples=GSM8K_FEWSHOT_EXAMPLES_LATS_REFLECT,
    value_examples=GSM8K_FEWSHOT_EXAMPLES_LATS_VALUE,
    prompt=LATS_INSTRUCTION_GSM8K,
    reflect_prompt=LATS_REFLECT_INSTRUCTION_GSM8K,
    value_prompt=LATS_VALUE_INSTRUCTION_GSM8K,            
    additional_keys={},
    reflect_additional_keys={},
    value_additional_keys={},
    max_iterations=1,
    reset=True
)

print(get_node_trajectory_math(out.answer))

In [3]:
import importlib
from agential.cog.lats.strategies import code

# Reload the module
importlib.reload(code)

# Now use the updated module
from agential.cog.lats.strategies.code import LATSCodeStrategy


In [4]:
inst = {"task_id": "HumanEval/0", "prompt": "from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    \"\"\" Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    \"\"\"\n", "entry_point": "has_close_elements", "canonical_solution": "    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False\n", "test": "\n\nMETADATA = {\n    'author': 'jt',\n    'dataset': 'test'\n}\n\n\ndef check(candidate):\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False\n    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True\n    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.8) == False\n    assert candidate([1.0, 2.0, 3.0, 4.0, 5.0, 2.0], 0.1) == True\n    assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 1.0) == True\n    assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 0.5) == False\n\n"}
question = inst['prompt']
key = f"{inst['test']}\ncheck({inst['entry_point']})"

agent = LATSCodeStrategy(
    llm=llm, 
    n_samples=2, 
    max_reflections=4, 
    depth_limit=5,
    max_unique=5,
    cache_values=True,
)

out = agent.generate(
    question=question,
    key=key,
    examples=HUMANEVAL_FEWSHOT_EXAMPLES_REACT,
    reflect_examples=HUMANEVAL_FEWSHOT_EXAMPLES_LATS_REFLECT,
    value_examples=HUMANEVAL_FEWSHOT_EXAMPLES_LATS_VALUE,
    prompt=LATS_INSTRUCTION_HUMANEVAL,
    reflect_prompt=LATS_REFLECT_INSTRUCTION_HUMANEVAL,
    value_prompt=LATS_VALUE_INSTRUCTION_HUMANEVAL,            
    additional_keys={},
    reflect_additional_keys={},
    value_additional_keys={},
    max_iterations=1,
    reset=True
)

print(get_node_trajectory_code(out.answer))

IM HERE


AttributeError: 'tuple' object has no attribute 'answer'

: 

In [ ]:
out

In [ ]:
question = "Write a python function to find the first repeated character in a given string."
key = """assert first_repeated_char("abcabc") == "a"
assert first_repeated_char("abc") == None
assert first_repeated_char("123123") == "1\""""

agent = LATSCodeStrategy(
    llm=llm, 
    n_samples=2, 
    max_reflections=4, 
    depth_limit=5,
    max_unique=5,
    cache_values=True,
)

out = agent.generate(
    question=question,
    key=key,
    examples=MBPP_FEWSHOT_EXAMPLES_REACT,
    reflect_examples=MBPP_FEWSHOT_EXAMPLES_LATS_REFLECT,
    value_examples=MBPP_FEWSHOT_EXAMPLES_LATS_VALUE,
    prompt=LATS_INSTRUCTION_MBPP,
    reflect_prompt=LATS_REFLECT_INSTRUCTION_MBPP,
    value_prompt=LATS_VALUE_INSTRUCTION_MBPP,            
    additional_keys={"tests": key},
    reflect_additional_keys={"tests": key},
    value_additional_keys={"tests": key},
    max_iterations=1,
    reset=True
)

print(get_node_trajectory_code(out.answer))

In [ ]:
out